In [1]:
import brightway2 as bw
import bw2regional as bwr
import bw2_lcimpact
from copy import copy
import constructive_geometries
import os
from bw2regional.pandarus_remote import AlreadyExists

In [2]:
assert bwr.__version__ >= (0, 5, 2)
assert bw2_lcimpact.__version__ >= (0, 3, 1)

In [ ]:
bw.projects.set_current("EXIOBASE regionalized case study")

In [ ]:
bw.databases

In [ ]:
bw.methods

In [ ]:
bwr.bw2regionalsetup('world')

In [ ]:
bwr.geocollections['world']

We need to define the Rest-of-Worlds used in EXIOBASE. Happily, they are only a few of them:

* **WA**:	RoW Asia and Pacific
* **WL**:	RoW America
* **WE**:	RoW Europe
* **WF**:	RoW Africa - really unfortunate acronym, as this is the code for an actual location...
* **WM**:	RoW Middle East

We will label the EXIOBASE Rest-of-Worlds with a) a new `Geocollection`, and b) more suitable names.

In [ ]:
exiobase_row_mapping = {
    "WA": "RoW Asia and Pacific",
    "WL": "RoW America",
    "WE": "RoW Europe",
    "WF": "RoW Africa",
    "WM": "RoW Middle East",
}

In [ ]:
bwr.geocollections['exiobase-rows'] = {
    'filepath': os.path.join(os.getcwd(), "exiobase-rows.gpkg"),
    'field': 'name'
}

In [ ]:
for activity in bw.Database("EXIOBASE 3.3.17 hybrid"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In [ ]:
for activity in bw.Database("Oil seeds"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In [ ]:
bw.databases['biosphere3']['geocollections'] = []
bw.databases['EXIOBASE 3.3.17 hybrid']['geocollections'] = ['world', 'exiobase-rows']
bw.databases['Oil seeds']['geocollections'] = ['world', 'exiobase-rows']
bw.databases.flush()

In a separate noteboom, created GIS data for these "rest-of-world" locations.

<img src="images/exiobase-row.png">

In [ ]:
bw2_lcimpact.import_global_lcimpact()

In [ ]:
bw2_lcimpact.import_regionalized_lcimpact()

In [ ]:
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "particulate-matter", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "ecoregions", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-hh", 
)

In [ ]:
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
    'exiobase-rows-watersheds-eq-sw-certain'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
    'exiobase-rows-watersheds-eq-sw-all'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "particulate-matter", 
    'exiobase-rows-particulate-matter'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "ecoregions", 
    'exiobase-rows-ecoregions'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-hh", 
    'exiobase-rows-watersheds-hh'
)

In [ ]:
from bw2regional import create_empty_intersection
# create_empty_intersection(('world', 'exiobase-rows-ecoregions'))
# create_empty_intersection(('exiobase-rows', 'ecoregions'))
# bwr.remote.calculate_intersection('exiobase-rows', 'world-topo-ecoregions')

In [ ]:
sorted(bwr.intersections)

In [ ]:
other_scales = [
    'world-topo-watersheds-eq-sw-all',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-hh',
    'world-topo-particulate-matter',
    'world-topo-ecoregions',    
]

for gc in other_scales:
    bwr.remote.intersection('exiobase-rows', gc)

# Where are crops actually grown?

Although this data source is not perfect, [Ramankutty et al](http://www.earthstat.org/harvested-area-yield-175-crops/) provide data on all our crops (note that peanuts can also be called groundnuts, as they are here).

I downloaded this data, and created compressed GeoTIFFS with `gdal_translate`:

    gdal_translate input_filename output_filename -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=DEFLATE

We also needed to transform the production rasters from total production per raster cell, to intensity (i.e. production per area, but note that this is total area, not harvested area). There is a function in `bw2regional` to do this already:

    bwr.divide_by_area(source, destination)


In [ ]:
rasters = (
    ("gdpweighted.tiff", "weighted-pop-density"),
    ("oilpalm_intensity.tiff", "oilpalm"),
    ("rapeseed_intensity.tiff", "rapeseed"),
    ("soybean_intensity.tiff", "soybean"),
    ("groundnut_intensity.tiff", "peanut"),
    ("sunflower_intensity.tiff", "sunflower"),
)

In [ ]:
third_scales = [
    'exiobase-rows-ecoregions',
    'exiobase-rows-particulate-matter',
    'exiobase-rows-watersheds-eq-sw-all',
    'exiobase-rows-watersheds-eq-sw-certain',
    'exiobase-rows-watersheds-hh',
    'world-topo-ecoregions',
    'world-topo-particulate-matter',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-hh'
]

In [ ]:
for x, y in rasters:
    bwr.geocollections[y] = {
        'filepath': os.path.abspath("data/" + x),
        'band': 1
    }

In [ ]:
for fp, raster in rasters:
    for gc in third_scales:
        bwr.remote.rasterstats_as_xt(gc, raster, f"xt-{raster}-{gc}")